Use weather API to get historic data: https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx

In [ ]:
import requests
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from random import randint
from time import sleep
import pickle
from os import makedirs
from os.path import join, exists

In [ ]:
save_folder = 'drive/My Drive/Colab Notebooks/Wishmart/weather_data/'
stores_data_file = 'drive/My Drive/Colab Notebooks/Wishmart/data/stores.csv'

In [ ]:
stores_df = pd.read_csv(stores_data_file)
unique_cities = list(stores_df['city'].unique())
unique_cities = ["{},ecuador".format(c.lower().replace(' ','+')) for c in unique_cities]
print(len(unique_cities), unique_cities)

22 ['quito,ecuador', 'santo+domingo,ecuador', 'cayambe,ecuador', 'latacunga,ecuador', 'riobamba,ecuador', 'ibarra,ecuador', 'guaranda,ecuador', 'puyo,ecuador', 'ambato,ecuador', 'guayaquil,ecuador', 'salinas,ecuador', 'daule,ecuador', 'babahoyo,ecuador', 'quevedo,ecuador', 'playas,ecuador', 'libertad,ecuador', 'cuenca,ecuador', 'loja,ecuador', 'machala,ecuador', 'esmeraldas,ecuador', 'manta,ecuador', 'el+carmen,ecuador']


In [ ]:
begin_data = datetime.datetime(2013, 1, 1)
end_date = datetime.datetime(2017, 8, 16)

In [ ]:
date_list = []
curr_date = begin_data
while curr_date < end_date:
  date_list.append((curr_date, curr_date + relativedelta(months=+1)))
  curr_date = curr_date + relativedelta(months=+1)
len(date_list)

56

In [ ]:
# api-endpoint 
URL = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx"

for city in unique_cities[:8]:
  for begin, end in date_list:

    out_name = 'w_{}_{}_{}'.format(city, begin.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))

    # check if already downloaded
    if exists(join(save_folder, '{}.csv'.format(out_name))):
      continue

    # Random timer
    sleep(randint(10, 20))

    print(city, begin, end)

    params = {
        'q': city,
        'date': begin.strftime("%Y-%m-%d"),
        'enddate' : end.strftime("%Y-%m-%d"),
        'format' : 'json',
        'key': '9770764302a947daae085734200308'
    }
    
    # sending get request and saving the response as response object
    try:
      r = requests.get(url = URL, params = params)
    except:
      print('Error with ', out_name)
      continue
      
    # extracting data in json format 
    try:
      result_data = r.json()
    except:
      print('Error with ', out_name)
      continue

    weather_data_list = []
    for days_weather in result_data['data']['weather']:
      day = days_weather['date']
      maxtempC = days_weather['maxtempC']
      avgtempC = days_weather['avgtempC']
      weather_data_list.append([day, maxtempC, avgtempC])

    # Save as DF
    df_temp = pd.DataFrame(weather_data_list, columns=['Date', 'MaxTemp', 'AvgTemp'])
    df_temp.to_csv(join(save_folder, "{}.csv".format(out_name)), index=None)

    # Save as pickle
    with open(join(save_folder, "{}.pickle".format(out_name)), 'wb') as handle:
      pickle.dump(result_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Clean variables
    del df_temp
    del result_data
    del weather_data_list
    del r

puyo,ecuador 2015-02-01 00:00:00 2015-03-01 00:00:00


## Combine all files into a single CSV

In [ ]:
in_folder = save_folder[:]
out_file = 'drive/My Drive/Colab Notebooks/Wishmart/data/weather.csv'

In [ ]:
all_files = [join(in_folder, f) for f in listdir(in_folder) if '.csv' in f]
len(all_files)

1232

In [ ]:
df_final = None

for f in tqdm(all_files):
  df = pd.read_csv(f)
  df['location'] = f.split('/')[-1].split(',')[0].split('_')[-1].replace('+', ' ')

  if df_final is None:
    df_final = df
  else:
    df_final = df_final.append(df, ignore_index=True)

100%|██████████| 1232/1232 [00:08<00:00, 137.06it/s]


In [ ]:
df_final.groupby(['location'])['Date'].count()

location
ambato           1760
babahoyo         1760
cayambe          1760
cuenca           1760
daule            1760
el carmen        1760
esmeraldas       1760
guaranda         1760
guayaquil        1760
ibarra           1760
latacunga        1760
libertad         1760
loja             1760
machala          1760
manta            1760
playas           1760
puyo             1760
quevedo          1760
quito            1760
riobamba         1760
salinas          1760
santo domingo    1760
Name: Date, dtype: int64

In [ ]:
df_final.to_csv(out_file, index=None)